In [0]:
%run "../../config/project_config"

In [0]:
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql.types import TimestampType

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {db_silver}")
print(f"DB silver hazır{db_silver}")


In [0]:
# Cell 2: Silver Dönüşüm Fonksiyonu (Düzeltilmiş - ANSI Mode Kapalı)

def transform_silver_layer():
    # 1. AYAR: Hata verip durmak yerine, bozuk veriyi NULL yapmasını söylüyoruz.
    spark.conf.set("spark.sql.ansi.enabled", "false")
    
    print(f"🚀 Silver Layer Dönüşümü Başlıyor... Hedef: {db_silver}\n")

    date_columns_map = {
        "orders": [
            "order_purchase_timestamp", 
            "order_approved_at", 
            "order_delivered_carrier_date", 
            "order_delivered_customer_date", 
            "order_estimated_delivery_date"
        ],
        "order_items": ["shipping_limit_date"],
        "order_reviews": ["review_creation_date", "review_answer_timestamp"]
    }

    for table_name in datasets.keys():
        try:
            df = spark.read.table(f"{db_bronze}.{table_name}")
            
            if table_name in date_columns_map:
                cols_to_fix = date_columns_map[table_name]
                
                print(f"🛠️ {table_name}: Tarih formatları düzeltiliyor -> {cols_to_fix}")
                
                for col_name in cols_to_fix:
                    # ANSI modu kapalı olduğu için, bozuk veri gelirse NULL atar, patlamaz.
                    df = df.withColumn(col_name, to_timestamp(col(col_name)))
            
            target_table = f"{db_silver}.{table_name}"
            
            df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(target_table)
            
            print(f"✅ {target_table} oluşturuldu.")
            
        except Exception as e:
            print(f"❌ HATA ({table_name}): {e}")

    print("\n🎉 Silver Katmanı Dönüşümü Tamamlandı!")

# Fonksiyonu çalıştır
transform_silver_layer()